In [1]:
from groq import Groq
import os
import datetime
import subprocess
import time
from dotenv import load_dotenv
import json

load_dotenv()
api_key = os.getenv('GROQ_API_KEY')
client = Groq(api_key=api_key)

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_time': 0.021666667, 'prompt_time': 0.002298616, 'queue_time': 0.011478854, 'total_time': 0.023965283}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-904989d9-c694-4aa3-8c1a-37f1b25a6757-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41})

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm happy to help! However, I'm a large language model, I don't have the ability to know or remember your name. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. So, I don't have your name to tell you. Would you like to introduce yourself?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 15, 'total_tokens': 86, 'completion_time': 0.059166667, 'prompt_time': 0.002920521, 'queue_time': 0.013163669000000001, 'total_time': 0.062087188}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-7e5cdcb8-9e74-4516-9c8a-144cecc21c3b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 71, 'total_tokens': 86})

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 40, 'total_tokens': 46, 'completion_time': 0.005, 'prompt_time': 0.013522623, 'queue_time': 0.113323887, 'total_time': 0.018522623}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-bdf86e84-f4b5-4b64-ba97-b340aa25d9f4-0', usage_metadata={'input_tokens': 40, 'output_tokens': 6, 'total_tokens': 46})

In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START,MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [5]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [6]:
output['messages']

[HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='530c2991-8f45-41c8-98eb-dc7f69c6e35b'),
 AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 16, 'total_tokens': 42, 'completion_time': 0.021666667, 'prompt_time': 0.000153169, 'queue_time': 0.013667231, 'total_time': 0.021819836}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-76553beb-08fe-4016-8e0c-d57f33e43015-0', usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42})]

In [7]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I remember! Your name is Bob!


In [8]:
output['messages']

[HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='530c2991-8f45-41c8-98eb-dc7f69c6e35b'),
 AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 16, 'total_tokens': 42, 'completion_time': 0.021666667, 'prompt_time': 0.000153169, 'queue_time': 0.013667231, 'total_time': 0.021819836}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-76553beb-08fe-4016-8e0c-d57f33e43015-0', usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42}),
 HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='2c8eae58-beb5-405c-ae79-1d4cfd89995a'),
 AIMessage(content='I remember! Your name is Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_toke

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert in answering questions related to user profile. You will be given with a resume in json and a job descriptipn. Your task is to answer the user related questions, to solve job queries",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [10]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [35]:
job_description = """
 About the job

Mercor is training models that predict job performance better than a human can. Our technology is so powerful that it’s trusted by customers ranging from startups making their first hires to the largest AI labs hiring thousands of experts to train LLMs.


We’re at an 8-figure revenue run rate, growing 50% MoM, and have raised $32M from investors like Benchmark, General Catalyst, Peter Thiel, Jack Dorsey, Adam D'Angelo, and Larry Summers.


---------------------------------------------------------------


Regardless of whether you have human data experience, we think that successful Machine Learning Engineers at Mercor should have a solid technical background, demonstrated through:


    2+ years of experience in back-end development (e.g., Python / Django)
    Strong understanding of database design, SQL, and commonly used algorithms
    Some experience with cloud platforms (GCP or AWS) and infrastructure management
    Experience working with LLMs and/or other transformer-based models such as BERT
    Experience with search or recommendation systems, data classification, or LTR models
    Able to quickly understand complex problems & rapidly develop minimum viable solutions
    Attention to detail, particularly in terms of producing high-quality, stable code that can support rapid scaling
    A strong belief in regularly testing and using the things you build, and a strong sense of ownership for everything you’ve developed





"""

In [36]:
import json

with open('../resume.json') as f:
    resume_json = json.load(f)

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START,MessagesState, StateGraph
from langchain_core.messages import HumanMessage, AIMessage


prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "You are a highly skilled assistant specialized in answering questions related to user profiles, particularly resumes and job descriptions. "
         "You will be provided with a resume (in JSON format) and a job description. Your task is to answer questions about the user's qualifications, "
         "provide insights about how well the resume aligns with the job description. "
         "You should be able to summarize the user's experience and answer questions in such a way that it aligns with the job description. "
         "Chances for the user to get selected increases."
        ),
        # Placeholder for user and AI messages
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Define the workflow
workflow = StateGraph(state_schema=MessagesState)

# Model invocation function
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    # Assuming 'model' is your pre-defined model object, like OpenAI or another API
    response = model.invoke(prompt)  # Replace with your actual model call
    return {"messages": response}

# Add workflow edges and nodes
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Memory management to save state during conversations
memory = MemorySaver()

# Compile the application with the memory checkpointer
app = workflow.compile(checkpointer=memory)

# Assuming you have a defined model (e.g., OpenAI, Langchain model)


In [ ]:

threads = {}

def handle_conversation(user_input, thread_id, resume=None, job_description=None):
    if thread_id not in threads or resume or job_description:
        # Initialize or reset the thread if needed
        threads[thread_id] = {
            "resume": resume,
            "job_description": job_description,
            "messages": [],
        }

        # Add the context to the conversation
        context = []
        if resume:
            context.append(HumanMessage(content=f"Here is the resume: {resume}"))
        if job_description:
            context.append(HumanMessage(content=f"Here is the job description: {job_description}"))
        
        # Add the user input as the latest message
        context.append(HumanMessage(content=user_input))
    else:
        # If thread exists and no new resume/job description, just add user input
        context = [HumanMessage(content=user_input)]

    # Assuming `app.invoke` is used to call the chatbot model (e.g., OpenAI API or another custom framework)
    config = {"configurable": {"thread_id": thread_id}}
    output = app.invoke({"messages": context}, config)
    threads[thread_id]["messages"] = output["messages"]
    return output["messages"][-1]

# Example usage
response = handle_conversation(
    user_input="Describe in a para and respect to first person, why i am a good fit for this job",
    thread_id="thread_1",
    resume=resume_json,
    job_description=job_description
)

# Assuming the output is a structured object with a `pretty_print()` method
print(response.pretty_print())


================================== Ai Message ==================================

As I review the job description, I am excited to see how my skills and experience align with the requirements. With over 3 years of experience in back-end development, I have a solid technical background that would enable me to hit the ground running. My proficiency in Python, Django, and SQL would allow me to tackle complex problems and rapidly develop minimum viable solutions. Additionally, my experience working with various cloud platforms, including Azure and AWS, would enable me to effectively manage infrastructure. I'm particularly drawn to this role because of Mercor's innovative approach to training models that predict job performance better than humans can. My experience with machine learning models, including TensorFlow, Keras, and PyTorch, would be a strong asset in this role. Furthermore, my ability to quickly understand complex problems and produce high-quality, stable code that can support r

In [46]:
response

AIMessage(content='As a Machine Learning Engineer with a strong technical background, I believe I would be a great fit for the role at Mercor. With my experience in back-end development, I am confident in my ability to design and implement scalable and efficient solutions. My understanding of database design, SQL, and algorithms would enable me to effectively manage and analyze large datasets. Additionally, my experience with cloud platforms and infrastructure management would allow', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 8111, 'total_tokens': 8193, 'completion_time': 0.068333333, 'prompt_time': 0.94821048, 'queue_time': 0.0017025290000000082, 'total_time': 1.016543813}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-b18b5224-603b-4404-bde7-a33e42a07b4e-0', usage_metadata={'input_tokens': 8111, 'output_tokens': 82, 'total_tokens': 8193})